/bin/bash: conda: command not found


In [1]:
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset,DataLoader,random_split
import torch.nn as nn

from torch.optim import Adam, SGD, lr_scheduler


### Read data from datalake

In [2]:
!pip3 install --upgrade pip

In [21]:
!pip3 install deeplake

In [22]:
!pip install --upgrade urllib3

In [24]:
!pip3 install -U deeplake

In [ ]:
!pip3 install --upgrade botocore


In [2]:
import deeplake
train_data = deeplake.load("hub://activeloop/gtsrb-train")
validation_data = deeplake.load("hub://activeloop/gtsrb-test")


/home/amber/anaconda3/envs/traffic_sign/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.6) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
/

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/gtsrb-train


\

...

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/gtsrb-test


/

hub://activeloop/gtsrb-test loaded successfully.


In [10]:
train_data[:100].summary()

Dataset(path='hub://activeloop/gtsrb-train', read_only=True, index=Index([slice(None, 100, None)]), tensors=['images', 'boxes', 'labels', 'shapes', 'colors'])

 tensor      htype              shape             dtype  compression
 -------    -------            -------           -------  ------- 
 images      image     (100, 25:168, 26:191, 3)   uint8    jpeg   
  boxes      bbox            (100, 1, 4)         float32   None   
 labels   class_label          (100, 1)          uint32    None   
 shapes   class_label          (100, 1)          uint32    None   
 colors   class_label          (100, 1)          uint32    None   


In [5]:
train_data.visualize()

HINT: Please forward the port - 56607 to your local machine, if you are running on the cloud.
 * Serving Flask app 'dataset_visualizer'
 * Debug mode: off


In [ ]:
# Indexing
image = train_data.images[:20000].numpy(aslist=True) # Fetch the first image and return a numpy array
max(i.shape for i in image)

(225, 243, 3)

In [ ]:
boxes = train_data.boxes[0].data()
boxes

{'value': array([[ 5.,  6., 16., 15.]], dtype=float32)}

In [ ]:
labels = train_data.labels[0].data() # Fetch the labels in the first image
labels

{'value': array([18], dtype=uint32), 'text': ['narrow_road_on_right']}

In [ ]:

# Slicing
img_list = train_data.labels[0:5].numpy(aslist=True) # Fetch 100 labels and store
                                               # them as a list of numpy arrays
img_list

[array([18], dtype=uint32),
 array([18], dtype=uint32),
 array([18], dtype=uint32),
 array([18], dtype=uint32),
 array([18], dtype=uint32)]

In [ ]:
shape = train_data.shapes[:5].data() # Fetch the labels in the first image
shape

{'value': array([[0],
        [0],
        [0],
        [0],
        [0]], dtype=uint32),
 'text': [['triangle'],
  ['triangle'],
  ['triangle'],
  ['triangle'],
  ['triangle']]}

In [ ]:
colors = train_data.colors[:5].data() # Fetch the labels in the first image
colors

{'value': array([[0],
        [0],
        [0],
        [0],
        [0]], dtype=uint32),
 'text': [['red'], ['red'], ['red'], ['red'], ['red']]}

In [ ]:
labels_list = train_data.labels.info['class_names']
labels_list[:5]

['pedestrian',
 'speed_limit_20km/hr',
 'speed_limit_30km⁄hr',
 'no_passing_of _vehicle_over_3.5ton',
 'crossroads']

### Transform

In [3]:
# from torchvision.transforms.v2 import ToTensor,Resize,Compose,ColorJitter,RandomRotation,AugMix,RandomCrop,GaussianBlur,RandomEqualize,RandomHorizontalFlip,RandomVerticalFlip
from torchvision.transforms import v2
from torchvision import transforms, models
# train_transforms = Compose([
#     ColorJitter(brightness=1.0, contrast=0.5, saturation=1, hue=0.1),
#     RandomEqualize(0.4),
#     AugMix(),
#     RandomHorizontalFlip(0.3),
#     RandomVerticalFlip(0.3),
#     GaussianBlur((3,3)),
#     RandomRotation(30),

#     Resize([50,50]),
#     ToTensor(),
    
# ])
# validation_transforms =  Compose([
#     Resize([50,50]),
#     ToTensor(),
    
# ])

tform = v2.Compose([
    v2.ToPILImage(),

    v2.ColorJitter(brightness=1.0, contrast=0.5, saturation=1, hue=0.1),
    v2.RandomEqualize(0.4),
    v2.AugMix(),
    v2.RandomHorizontalFlip(p=0.3),
    v2.RandomVerticalFlip(0.3),
    v2.GaussianBlur((3,3)),
    v2.RandomRotation(30),
    v2.Resize(size=(50,50)),
    v2.ToTensor(),

])
validation_transforms =  v2.Compose([
    v2.ToPILImage(),

    v2.Resize(size=(50,50)),
    v2.ToTensor(),

])

/home/amber/anaconda3/envs/traffic_sign/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/amber/anaconda3/envs/traffic_sign/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Plea

In [ ]:
# from torch.utils.data import DataLoader, Dataset

# class ClassificationDataset(Dataset):
#     def __init__(self, ds, transform = tform):
#         self.ds = ds
#         self.images = ds.images
#         self.labels = ds.labels
#         self.transform = transform

#     def __len__(self):
#         return len(self.ds)

#     def __getitem__(self, idx):
#         image = self.ds.images[idx].numpy()
#         label = self.ds.labels[idx].numpy(fetch_chunks = True).astype(np.int32)

#         if self.transform is not None:
#             image = self.transform(image)
#         print(label)

#         sample = {"images": image, "labels": label}

#         return sample
# transformed_ds = ClassificationDataset(train_data, transform = tform)


In [ ]:
# data = transformed_ds.images[0].numpy(aslist=True)
# print(data.shape)

(26, 26, 3)


/usr/local/lib/python3.10/dist-packages/deeplake/core/tensor.py:675: UserWarning: Indexing by integer in a for loop, like `for i in range(len(ds)): ... ds.tensor[i]` can be quite slow. Use `for i, sample in enumerate(ds)` instead.
  warnings.warn(


### Dataloader

In [6]:
train_data.summary()

Dataset(path='hub://activeloop/gtsrb-train', read_only=True, tensors=['images', 'boxes', 'labels', 'shapes', 'colors'])

 tensor      htype               shape              dtype  compression
 -------    -------             -------            -------  ------- 
 images      image     (39209, 25:225, 25:243, 3)   uint8    jpeg   
  boxes      bbox            (39209, 1, 4)         float32   None   
 labels   class_label          (39209, 1)          uint32    None   
 shapes   class_label          (39209, 1)          uint32    None   
 colors   class_label          (39209, 1)          uint32    None   


In [4]:
def train_test_split(dataset,train_size):
    test_size = 1-train_size
    return dataset.random_split([train_size, test_size])

In [5]:
train_set,validation_set = train_test_split(train_data,0.8)
train_set.summary(force=True)

Dataset(path='hub://activeloop/gtsrb-train', read_only=True, index=Index([(3492, 8865, 5381, 12, 6775, 4339, 4610, 5235, 5765, 4775, 5373, 8957, 4481, 8614, 8936, 8391, 8744, 4117, 3445, 1638, 201, 8357, 5498, 1961, 349, 6321, 9742, 8946, 9544, 6980, 5745, 5237, 226, 7065, 7586, 8416, 8955, 10082, 5333, 10489, 6774, 9461, 9195, 8077, 9346, 5142, 2845, 4470, 477, 4747, 5668, 6296, 1186, 8999, 6200, 5916, 7729, 8324, 7587, 1848, 5872, 9367, 2731, 7972, 3064, 6886, 4132, 5483, 10030, 1365, 512, 10447, 9399, 10587, 8833, 10115, 8941, 4636, 9572, 3642, 1804, 1358, 9978, 5410, 5213, 5834, 3063, 10406, 7144, 4021, 1375, 3336, 4669, 3621, 6729, 966, 5880, 6563, 8048, 5265, 7742, 4377, 4231, 647, 7585, 405, 290, 6589, 5643, 2810, 9771, 7285, 4196, 9973, 3450, 9695, 6223, 9744, 350, 8351, 3979, 6457, 9559, 2568, 10185, 3922, 441, 1563, 3022, 2361, 6552, 5694, 9403, 5456, 1679, 6762, 9631, 3173, 4189, 1656, 6025, 10445, 2471, 8948, 9404, 8964, 3736, 545, 1562, 7601, 7623, 6935, 141, 7670, 5382, 3

In [12]:
validation_set.summary(force=True)

Dataset(path='hub://activeloop/gtsrb-train', read_only=True, index=Index([(6750, 5465, 2322, 5704, 4982, 2051, 5042, 632, 2150, 3179, 3443, 1708, 9777, 7469, 1050, 8669, 1974, 9101, 6349, 1595, 9872, 565, 3184, 5430, 2719, 8833, 6176, 5524, 829, 6317, 7277, 2629, 4084, 4480, 5013, 8326, 1659, 2226, 1387, 5194, 5683, 8929, 5750, 9678, 4556, 6210, 779, 10343, 6128, 1947, 8487, 2919, 3211, 8810, 3346, 3087, 6580, 10120, 2723, 8858, 10525, 7254, 8341, 7263, 10165, 6978, 6134, 9320, 995, 275, 7610, 9846, 9046, 5657, 5565, 9039, 6763, 9417, 10028, 8657, 633, 819, 5287, 5113, 1278, 4688, 4963, 3900, 5278, 6047, 8471, 4943, 3504, 1525, 6883, 4790, 3498, 4740, 4622, 7845, 9900, 5952, 4597, 7285, 1995, 10306, 2436, 4583, 2021, 3921, 9681, 692, 206, 799, 1767, 8584, 5170, 4335, 4248, 2649, 9145, 5604, 5979, 442, 7675, 3287, 4279, 1620, 10261, 1970, 3674, 2446, 1880, 3398, 7533, 3987, 940, 5632, 7585, 6492, 10151, 4839, 4567, 9343, 1894, 7161, 492, 3801, 4443, 2677, 6082, 7624, 9429, 8696, 3067, 7

In [6]:
BATCH_SIZE = 64
training_loader = train_set.pytorch(num_workers=8, batch_size=BATCH_SIZE, decode_method = {'images': 'pil'}, transform={'images': tform,'boxes': None, 'labels': None, 'shapes': None, 'colors': None}, shuffle=False)
validation_loader = validation_set.pytorch(num_workers=8, batch_size=BATCH_SIZE, decode_method = {'images': 'pil'}, transform={'images': validation_transforms,'boxes': None, 'labels': None, 'shapes': None, 'colors': None}, shuffle=False)

In [24]:
train_features, boxs, train_labels, shapes, colors = next(iter(training_loader))

print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img.T, cmap="gray")
plt.show()
print(f"Label: {label}")

Please wait, filling up the shuffle buffer with samples.:   2%|▏         | 40.4M/1.91G [02:46<1:54:28, 292kB/s]   

KeyboardInterrupt: 

### Modeling

In [10]:
from torchvision import models
import torch


# net = models.resnet18(weights=None)
# net.fc = torch.nn.Linear(net.fc.in_features, len(train_data.labels.info.class_names))

# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [101]:
# for epoch in range(2):
#     running_loss = 0.0
#     for i, data in enumerate(training_loader):
#         images, labels = data['images'], data['labels']

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = net(images)
#         loss = criterion(outputs, labels.reshape(-1))
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 100 == 99:    # print every 100 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                 (epoch + 1, i + 1, running_loss / 100))
#             running_loss = 0.0


Please wait, filling up the shuffle buffer with samples.:   0%|          | 0.00/1.91G [00:00<?, ?B/s]
Please wait, filling up the shuffle buffer with samples.:   0%|          | 29.3k/1.91G [00:31<598:35:11, 950B/s]
Please wait, filling up the shuffle buffer with samples.:   0%|          | 1.86M/1.91G [00:31<6:31:29, 87.1kB/s]
Please wait, filling up the shuffle buffer with samples.:   0%|          | 3.70M/1.91G [00:32<2:49:03, 202kB/s] 
Please wait, filling up the shuffle buffer with samples.:   0%|          | 5.53M/1.91G [00:32<1:33:27, 364kB/s]
Please wait, filling up the shuffle buffer with samples.:   0%|          | 7.36M/1.91G [00:33<1:01:32, 553kB/s]
Please wait, filling up the shuffle buffer with samples.:   0%|          | 9.20M/1.91G [00:33<40:29, 839kB/s]  
Please wait, filling up the shuffle buffer with samples.:   1%|          | 11.0M/1.91G [00:34<30:29, 1.11MB/s]
Please wait, filling up the shuffle buffer with samples.:   1%|          | 12.9M/1.91G [00:34<21:27, 1.58MB/s]


Shuffle buffer filling is complete.
[1,   100] loss: 3.181
[1,   200] loss: 2.840
[1,   300] loss: 2.605
[1,   400] loss: 2.379



Please wait, filling up the shuffle buffer with samples.:   0%|          | 0.00/1.91G [00:00<?, ?B/s]
Please wait, filling up the shuffle buffer with samples.:   0%|          | 29.3k/1.91G [00:05<96:21:39, 5.90kB/s]
Please wait, filling up the shuffle buffer with samples.:   0%|          | 3.70M/1.91G [00:05<39:13, 869kB/s]    
Please wait, filling up the shuffle buffer with samples.:   0%|          | 7.36M/1.91G [00:06<20:42, 1.64MB/s]
Please wait, filling up the shuffle buffer with samples.:   0%|          | 9.20M/1.91G [00:06<15:15, 2.23MB/s]
Please wait, filling up the shuffle buffer with samples.:   1%|          | 11.0M/1.91G [00:07<14:10, 2.39MB/s]
Please wait, filling up the shuffle buffer with samples.:   1%|          | 12.9M/1.91G [00:07<11:09, 3.04MB/s]
Please wait, filling up the shuffle buffer with samples.:   1%|          | 14.7M/1.91G [00:08<10:59, 3.08MB/s]
Please wait, filling up the shuffle buffer with samples.:   1%|          | 16.5M/1.91G [00:08<09:02, 3.74MB/s]
Ple

Shuffle buffer filling is complete.
[2,   100] loss: 2.147
[2,   200] loss: 2.019
[2,   300] loss: 1.906
[2,   400] loss: 1.819


In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on the GPU")
else:
    device = torch.device("cpu")
    print("running on the CPU")

running on the GPU


In [112]:
len(training_loader)

429

In [8]:
import torch 
import torch.nn as nn
import random
import numpy as np
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

In [9]:
set_seed(42)

In [10]:
class GTSRB_MODEL(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(GTSRB_MODEL,self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim


        self.metrics = {}

        self.flatten = nn.Flatten()

        self.dropout2 = nn.Dropout(0.2)
        self.dropout3 = nn.Dropout(0.3)


        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(2)


        self.conv1 = nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1)
        self.batchnorm1 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1)
        self.batchnorm2 = nn.BatchNorm2d(256)


        self.conv5 = nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3)
        self.conv6 = nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=3)
        self.batchnorm3 = nn.BatchNorm2d(1024)


        self.l1 = nn.Linear(1024*4*4,512)
        self.l2 = nn.Linear(512,128)
        self.batchnorm4 = nn.LayerNorm(128)
        self.l3 = nn.Linear(128,output_dim)


    def forward(self,input):

        conv = self.conv1(input)
        conv = self.conv2(conv)
        batchnorm = self.relu(self.batchnorm1(conv))
        maxpool = self.maxpool(batchnorm)

        conv = self.conv3(maxpool)
        conv = self.conv4(conv)
        batchnorm = self.relu(self.batchnorm2(conv))
        maxpool = self.maxpool(batchnorm)

        conv = self.conv5(maxpool)
        conv = self.conv6(conv)
        batchnorm = self.relu(self.batchnorm3(conv))
        maxpool = self.maxpool(batchnorm)


        flatten = self.flatten(maxpool)

        dense_l1 = self.l1(flatten)
        dropout = self.dropout3(dense_l1)
        dense_l2 = self.l2(dropout)
        batchnorm = self.batchnorm4(dense_l2)
        dropout = self.dropout2(batchnorm)
        output = self.l3(dropout)


        return output

    def training_metrics(self,positives,data_size,loss):
        acc = positives/data_size
        return loss,acc

    def validation_metrics(self,validation_data,loss_function):
       data_size = len(validation_data)
       correct_predictions = 0
       total_samples = 0
       val_loss = 0

       model = self.eval()
       with torch.no_grad() :
        for step,validation_ in enumerate(validation_data):
            input,label = validation_['images'].to(device),validation_['labels'].to(device)
            label = label.reshape(-1)

            prediction = model.forward(input)
            loss = loss_function(prediction,label)
            val_loss = loss.item()
            _,predicted = torch.max(prediction,1)
            correct_predictions += (predicted == label).sum().item()
            total_samples += label.size(0)

       val_acc = correct_predictions/total_samples

       return val_loss,val_acc

    def history(self):
        return self.metrics


    def train_model(self,train_data,validation_data, device, epochs,loss_function,optimizer, learning_rate_scheduler):
        val_acc_list = []
        val_loss_list = []
        # best_acc = 0

        train_acc_list = []
        train_loss_list = []

        learning_rate_list = []

        print('training started ...')
        STEPS = len(train_data)
        for epoch in range(epochs):
            lr = optimizer.param_groups[0]["lr"]
            learning_rate_list.append(lr)
            correct_predictions = 0
            total_examples = 0
            loss = 0
            with tqdm.trange(STEPS) as progress:

                for step, train_ in enumerate(train_data):

                    input,label = train_['images'].to(device),train_['labels'].to(device)
                    labels = label.reshape(-1)
                    prediction = self.forward(input)

                    _, predicted = torch.max(prediction, 1)
                    correct_predictions += (predicted == labels).sum().item()
                    total_examples += labels.size(0)
                    l = loss_function(prediction,labels)
                    loss = l.item()
                    l.backward()
                    optimizer.step()
                    optimizer.zero_grad()
                    
                    # learning_rate_scheduler.step()
                    
                    

                    progress.colour = 'green'
                    progress.desc = f'Epoch [{epoch}/{EPOCHS}], Step [{step}/{STEPS}], Learning Rate [{lr}], Loss [{"{:.4f}".format(l)}], Accuracy [{"{:.4f}".format(correct_predictions/total_examples)}]'
                    progress.update(1)

            training_loss,training_acc = self.training_metrics(correct_predictions,total_examples,loss)
            train_acc_list.append(training_acc)
            train_loss_list.append(training_loss)

            val_loss, val_acc = self.validation_metrics(validation_data,loss_function)
            val_acc_list.append(val_acc)
            val_loss_list.append(val_loss)

            print(f'val_accuracy [{val_acc}], val_loss [{val_loss}]')

                
    

            learning_rate_scheduler.step()

        metrics_dict = {
                'train_acc':train_acc_list,
                'train_loss':train_loss_list,
                'val_acc':val_acc_list,
                'val_loss':val_loss_list,
                'learning_rate':optimizer.param_groups[0]["lr"]
            }
        self.metrics = metrics_dict
        print('training complete !')


In [11]:
import tqdm
EPOCHS = 10
LEARNING_RATE = 0.0008
INPUT_DIM = 3*50*50
OUTPUT_DIM = 43
model = GTSRB_MODEL(INPUT_DIM,OUTPUT_DIM).to(device)

optimizer = Adam(params=model.parameters(),lr=LEARNING_RATE)
# optimizer = SGD(model.parameters(), lr=1)
# scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.1)
lr_s = lr_scheduler.LinearLR(optimizer,start_factor=1.0,end_factor=0.5,total_iters=10)
loss = nn.CrossEntropyLoss()


In [12]:
model.train_model(train_data=training_loader,
                  validation_data=validation_loader,
                  device = device,
                  epochs=EPOCHS,
                  loss_function=loss,
                  optimizer=optimizer,
                  learning_rate_scheduler=lr_s)
                  # learning_rate_scheduler=scheduler)
# 


training started ...


Epoch [0/10], Step [490/491], Learning Rate [0.0008], Loss [1.4359], Accuracy [0.4562]: 100%|██████████| 491/491 [02:55<00:00,  2.80it/s]


val_accuracy [0.38898099732177016], val_loss [8.327280044555664]


Epoch [1/10], Step [490/491], Learning Rate [0.00076], Loss [0.7643], Accuracy [0.6211]: 100%|██████████| 491/491 [02:40<00:00,  3.07it/s]


val_accuracy [0.47800025506950644], val_loss [9.086514472961426]


Epoch [2/10], Step [490/491], Learning Rate [0.00072], Loss [0.9691], Accuracy [0.7016]: 100%|██████████| 491/491 [02:39<00:00,  3.08it/s]


KeyboardInterrupt: 

In [18]:
# saving a checkpoint assuming the network class named ClassNet
checkpoint = {
            'epoch': EPOCHS,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
    
            }

torch.save(checkpoint, 'checkpoint.pkl')



In [19]:
# Loading model
gtsrb_model = GTSRB_MODEL(3*50*50, 43)
def load_checkpoint(model, filepath):
    checkpoint = torch.load(filepath)
    epoch = checkpoint['epoch']  # 提取网络结构
    model.load_state_dict(checkpoint['model_state_dict'])  # 加载网络权重参数

    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])  # 加载优化器参数
    loss = checkpoint['loss']  # 提取网络结构

    for parameter in model.parameters():
        parameter.requires_grad = False
    model.eval()

    return model, loss

gtsrb_model, loss_function = load_checkpoint(gtsrb_model, 'checkpoint.pkl')

In [25]:
BATCH_SIZE = 64
from torchvision.transforms import v2
validation_transforms =  v2.Compose([
    v2.ToPILImage(),

    v2.Resize(size=(50,50)),
    v2.ToTensor(),

])
test_loader = validation_data.pytorch(num_workers=8, batch_size=BATCH_SIZE, decode_method = {'images': 'pil'}, transform={'images': validation_transforms,'boxes': None, 'labels': None, 'shapes': None, 'colors': None}, shuffle=False)
print("test dataset size:", len(test_loader))

/home/amber/anaconda3/envs/traffic_sign/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


test dataset size: 115


In [26]:
from sklearn.metrics import accuracy_score

y_pred = []
y_true = []

# correct_predictions = 0
# total_samples = 0
# gtsrb = gtsrb_model.to(device)
with tqdm.tqdm(colour='red',total=len(test_loader)) as progress:
  
  with torch.no_grad() : 
    for step,validation_ in enumerate(test_loader):
        input,label = validation_['images'].to(device),validation_['labels'].to(device)
        label = label.reshape(-1).cpu().numpy()
        
        
        prediction = model.forward(input)
        
        # loss = loss_function(prediction,label)
        # val_loss = loss.item()
        
        _,predicted = torch.max(prediction,1)
        predicted = predicted.cpu().numpy()
        # y_pred.append(predicted)
        
        # correct_predictions += (predicted == label).sum().item()
        # total_samples += label.size(0)
        # val_acc = correct_predictions/total_samples
                
        progress.desc = f'Test Accuracy:{accuracy_score(label,predicted)}'
        progress.update(1)

Test Accuracy:0.5573770491803278: 100%|██████████| 115/115 [00:21<00:00,  5.39it/s]
